<a href="https://colab.research.google.com/github/williamedwardhahn/AEye/blob/main/MPCR_Face_Tracking1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPCR Face Tracking Team

In [ ]:
pip install facenet-pytorch

In [ ]:
from facenet_pytorch import MTCNN
from PIL import Image
import torch
import cv2
import time
import glob
from tqdm.notebook import tqdm
import numpy as np
import imageio
import matplotlib.pyplot as plt
import matplotlib.patches as patches

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class FastMTCNN(object):
    """Fast MTCNN implementation."""
    
    def __init__(self, stride, resize=1, *args, **kwargs):
        """Constructor for FastMTCNN class.
        
        Arguments:
            stride (int): The detection stride. Faces will be detected every `stride` frames
                and remembered for `stride-1` frames.
        
        Keyword arguments:
            resize (float): Fractional frame scaling. [default: {1}]
            *args: Arguments to pass to the MTCNN constructor. See help(MTCNN).
            **kwargs: Keyword arguments to pass to the MTCNN constructor. See help(MTCNN).
        """
        self.stride = stride
        self.resize = resize
        self.mtcnn = MTCNN(*args, **kwargs)
        
    def __call__(self, frame):
        """Detect faces in frames using strided MTCNN."""
        # if self.resize != 1:
        #     frames = [
        #         cv2.resize(f, (int(f.shape[1] * self.resize), int(f.shape[0] * self.resize)))
        #             for f in frames
        #     ]
                      
        boxes, probs = self.mtcnn.detect(frame)

        # faces = []
        #     if boxes[box_ind] is None:
        #         continue
        #     for box in boxes[box_ind]:
        #         box = [int(b) for b in box]
        #         faces.append(frame[box[1]:box[3], box[0]:box[2]])
        
        return boxes

In [ ]:
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=1,
    margin=14,
    factor=0.6,
    keep_all=True,
    device=device
)

In [ ]:
def plot(x):
    fig, ax = plt.subplots()
    im = ax.imshow(x,cmap='gray')
    ax.axis('off')
    fig.set_size_inches(20, 20)
    plt.show()

In [ ]:
im = imageio.imread('https://cdn.psychologytoday.com/sites/default/files/styles/article-inline-half-caption/public/field_blog_entry_images/2018-09/shutterstock_648907024.jpg')[:,:,:3]

In [ ]:
plot(im)

In [ ]:
%%timeit
boxes = fast_mtcnn(im)

In [ ]:
boxes = fast_mtcnn(im)

In [ ]:
boxes

In [ ]:
plt.imshow(im)

In [ ]:
x1,y1,x2,y2 = np.array(boxes[0]).astype(int)
w = x2 - x1
h = y2 - y1

img = im[y1:y1+h,x1:x1+w] 

In [ ]:
plot(img)

In [ ]:
s = img.shape[0]//6

img = img[2*s:3*s]

In [ ]:
plot(img)

In [ ]:
# Live Demo

In [ ]:
from PIL import Image
import base64
from io import BytesIO
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
    //   await new Promise((resolve) => capture.onclick = resolve);
      await new Promise(r => setTimeout(r, 2000));



      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])

  img = Image.open(BytesIO(binary))
  img = np.asarray(img)

  return img

In [ ]:
for i in range(3):
    img = take_photo()
    plot(img)

In [ ]:
# for i in range(3):
#     img = take_photo()

#     boxes = fast_mtcnn(img)

#     x1,y1,x2,y2 = np.array(boxes[0]).astype(int)
#     w = x2 - x1
#     h = y2 - y1

#     img = img[y1:y1+h,x1:x1+w]

#     s = img.shape[0]//6

#     img = img[2*s:3*s]

#     plot(img)

In [ ]:
img = take_photo()

In [ ]:
boxes = fast_mtcnn(img)

x1,y1,x2,y2 = np.array(boxes[0]).astype(int)
w = x2 - x1
h = y2 - y1

img = img[y1:y1+h,x1:x1+w]

s = img.shape[0]//6

img = img[2*s:3*s]

plot(img)